In [31]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# 更新されたExcelファイルを読み込む
updated_file_path = '231217_data.xlsx'

# Excel ファイルを読み込む（列名の余分な空白を除去）
df_updated = pd.read_excel(updated_file_path)
df_updated.columns = df_updated.columns.str.strip()

# 1-8行目のデータを用いて重回帰分析を行う
df_train = df_updated.iloc[0:8]  # 1-8行目のデータ
X_train = df_train[['X', 'Y', 'A', 'B']]  # 説明変数
Y_train = df_train['Z']  # 目的変数

# 2次の多項式特徴量を生成
# https://helve-blog.com/posts/python/scikit-learn-polynomialfeatures/
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly.fit_transform(X_train)

# 重回帰モデルの作成とフィッティング（多項式特徴量を使用）
poly_reg_model = LinearRegression()
poly_reg_model.fit(X_poly_train, Y_train)

# 重回帰モデルの係数と切片を取得
poly_coefficients = poly_reg_model.coef_
poly_intercept = poly_reg_model.intercept_

# 生成された多項式特徴量の名称を取得
feature_names = poly.get_feature_names_out(['X', 'Y', 'A', 'B'])

# 係数と特徴量の名称を組み合わせて関係式を作成
relation_equation = "Predicted Z = " + str(poly_intercept)
for coef, name in zip(poly_coefficients, feature_names):
    relation_equation += f" + ({coef}) * {name}"

# 全てのデータに対して予測を行う
X_poly = poly.transform(df_updated[['X', 'Y', 'A', 'B']])
predicted_Z_poly = poly_reg_model.predict(X_poly)

# 予測値と実際の値の誤差を計算
errors = abs(predicted_Z_poly - df_updated['Z'])

# 新しいデータフレームに結果を格納
df_results = df_updated.copy()
df_results['Predicted Z'] = predicted_Z_poly
df_results['Error'] = errors

# 結果と回帰式をExcelファイルとして保存
output_file_path = 'predicted_results.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    df_results.to_excel(writer, sheet_name='Sheet1', index=False)
    pd.DataFrame({'Regression Equation': [relation_equation]}).to_excel(writer, sheet_name='Sheet2', index=False)

print(f'Results and regression equation saved to {output_file_path}')


Results and regression equation saved to predicted_results.xlsx
